# Optimization pipeline

## Import libraries

In [33]:
import os
import pandas as pd
import json
import time
from tqdm import tqdm
import numpy as np
import gurobipy

from collections import defaultdict
from itertools import product

## Variables

In [34]:
PREPARED_DAY = 'day-99.json'
ENRICHED_DAY = 'enriched_' + PREPARED_DAY
DATA_PATH = 'data/enriched_train/'
PREPARED_DATA_PATH = 'data/prepared_train/'

## Data

### Jobs

In [35]:
with open(DATA_PATH + ENRICHED_DAY,'r') as f:
    j=json.load(f)
    
pd.json_normalize(j['ovens'])

,start_temp,working_temps,operations
0,1070,"[1190, 1030, 1000, 970, 1010, 1140, 1020, 990,...","[otzhig, kovka, nagrev]"
1,1060,"[1080, 1060, 960]","[otzhig, kovka, nagrev]"
2,1010,"[960, 1060, 1130, 1200, 950, 1240, 1210, 1010]","[otzhig, nagrev]"
3,1060,"[1050, 1160, 1070, 1010, 980, 1240, 1210, 1060]","[kovka, nagrev]"
4,1170,"[970, 1010, 1170, 1140, 1240, 1210, 1180]","[prokat, nagrev]"
...,...,...,...
260,970,"[1220, 1190, 1130, 1100, 1040, 1200, 950, 1080...","[kovka, nagrev]"
261,1190,"[1230, 1070, 1040, 1010, 1180, 1190]","[kovka, nagrev]"
262,1000,"[1030, 1000, 970, 1170, 1240, 1020, 990]","[prokat, nagrev]"
263,1150,"[1220, 1100, 1030, 1150]","[otzhig, otzhig, nagrev]"


In [36]:
def extract_job_meta (job):
    '''
    извлечение заголовков операций из заданий, для последующего сравнения с печами
    '''
    i=job.name
    t=job['temperature']
    names=set([x['name'] for x in job['operations']])
    head=np.array([t]+list(names))
    nlst=[(x['name'],x['timing']) for x in job['operations']]
    return head, [i]+nlst

In [37]:
jobs_head = pd.json_normalize(j['series']).apply(lambda job: extract_job_meta (job),axis=1)
jobs_head

0       ([1010, prokat, nagrev], [0, (nagrev, 132), (p...
1       ([1000, otzhig, nagrev], [1, (nagrev, 52), (ot...
2       ([980, prokat, nagrev], [2, (nagrev, 220), (pr...
3       ([1000, prokat, nagrev], [3, (nagrev, 232), (p...
4       ([1060, prokat, nagrev], [4, (nagrev, 140), (p...
                              ...                        
2856    ([1090, prokat, nagrev], [2856, (nagrev, 35), ...
2857    ([990, kovka, nagrev], [2857, (nagrev, 57), (k...
2858    ([1030, otzhig, nagrev], [2858, (nagrev, 162),...
2859    ([990, prokat, nagrev], [2859, (nagrev, 199), ...
2860    ([1020, otzhig, nagrev], [2860, (nagrev, 108),...
Length: 2861, dtype: object

In [38]:
jobs_head[0]

(array(['1010', 'prokat', 'nagrev'], dtype='<U21'),
 [0, ('nagrev', 132), ('prokat', 15)])

In [39]:
jobs = jobs_head.head(40)
# можно использовать .sample(n)

### Ovens data

In [40]:
def extract_oven_meta (oven):
    '''
    извлечение заголовков из печей, для последующего сравнения с заданиями
    '''
    i = oven.name
    t = oven['start_temp']
    names = oven['operations']
    head = np.array([t] + list(names))
    return head, i, oven['working_temps']

In [41]:
ovens_head=pd.json_normalize(j['ovens']).apply(lambda job: extract_oven_meta (job), axis=1)
ovens_head

0      ([1070, otzhig, kovka, nagrev], 0, [1190, 1030...
1      ([1060, otzhig, kovka, nagrev], 1, [1080, 1060...
2      ([1010, otzhig, nagrev], 2, [960, 1060, 1130, ...
3      ([1060, kovka, nagrev], 3, [1050, 1160, 1070, ...
4      ([1170, prokat, nagrev], 4, [970, 1010, 1170, ...
                             ...                        
260    ([970, kovka, nagrev], 260, [1220, 1190, 1130,...
261    ([1190, kovka, nagrev], 261, [1230, 1070, 1040...
262    ([1000, prokat, nagrev], 262, [1030, 1000, 970...
263    ([1150, otzhig, otzhig, nagrev], 263, [1220, 1...
264    ([1190, otzhig, nagrev], 264, [960, 1120, 1000...
Length: 265, dtype: object

In [42]:
# ovens_head.head(7) не меняем
ovens = ovens_head.head(7)
ovens

0    ([1070, otzhig, kovka, nagrev], 0, [1190, 1030...
1    ([1060, otzhig, kovka, nagrev], 1, [1080, 1060...
2    ([1010, otzhig, nagrev], 2, [960, 1060, 1130, ...
3    ([1060, kovka, nagrev], 3, [1050, 1160, 1070, ...
4    ([1170, prokat, nagrev], 4, [970, 1010, 1170, ...
5    ([990, otzhig, otzhig, nagrev], 5, [1120, 1000...
6    ([1130, kovka, nagrev], 6, [1120, 960, 1090, 1...
dtype: object

## Ovens and Series match

In [43]:
ovens

0    ([1070, otzhig, kovka, nagrev], 0, [1190, 1030...
1    ([1060, otzhig, kovka, nagrev], 1, [1080, 1060...
2    ([1010, otzhig, nagrev], 2, [960, 1060, 1130, ...
3    ([1060, kovka, nagrev], 3, [1050, 1160, 1070, ...
4    ([1170, prokat, nagrev], 4, [970, 1010, 1170, ...
5    ([990, otzhig, otzhig, nagrev], 5, [1120, 1000...
6    ([1130, kovka, nagrev], 6, [1120, 960, 1090, 1...
dtype: object

In [44]:
jobs

0     ([1010, prokat, nagrev], [0, (nagrev, 132), (p...
1     ([1000, otzhig, nagrev], [1, (nagrev, 52), (ot...
2     ([980, prokat, nagrev], [2, (nagrev, 220), (pr...
3     ([1000, prokat, nagrev], [3, (nagrev, 232), (p...
4     ([1060, prokat, nagrev], [4, (nagrev, 140), (p...
5     ([970, otzhig, nagrev], [5, (nagrev, 213), (ot...
6     ([1040, kovka, nagrev], [6, (nagrev, 213), (ko...
7     ([970, prokat, nagrev], [7, (nagrev, 262), (pr...
8     ([980, otzhig, nagrev], [8, (nagrev, 247), (ot...
9     ([1060, prokat, nagrev], [9, (nagrev, 38), (pr...
10    ([1090, otzhig, nagrev], [10, (nagrev, 208), (...
11    ([950, kovka, nagrev], [11, (nagrev, 146), (ko...
12    ([1080, kovka, nagrev], [12, (nagrev, 191), (k...
13    ([1090, prokat, nagrev], [13, (nagrev, 156), (...
14    ([990, kovka, nagrev], [14, (nagrev, 265), (ko...
15    ([970, otzhig, nagrev], [15, (nagrev, 73), (ot...
16    ([1090, prokat, nagrev], [16, (nagrev, 130), (...
17    ([980, otzhig, nagrev], [17, (nagrev, 80),

In [46]:
matched_data = []

In [50]:
for job in jobs:
    job_to_list = [1, 0]
    sum_opp_time = 0
    
    for op_time in job[1][1:]:
        sum_opp_time += op_time[1]
        
    for oven in ovens:
        if np.all(np.isin(job[0], oven[0])):
            job_to_list[1] += 1
            job_to_list.append(oven[1]) 
            job_to_list.append(sum_opp_time)
        elif np.all(np.isin(job[0][1:], oven[0][1:])) and int(job[0][0]) in oven[2] and job_to_list[-2] != oven[1]:
            job_to_list[1] += 1
            job_to_list.append(oven[1]) 
            job_to_list.append(sum_opp_time + 240)
        if len(job_to_list) == 2:
            for oven in ovens:
                if np.all(np.isin(job[0][1:], oven[0][1:])):
                    job_to_list[1] += 1
                    job_to_list.append(oven[1]) 
                    job_to_list.append(sum_opp_time + 240)
                    break
    matched_data.append(job_to_list)

In [52]:
matched_data

[[1, 1, 4, 387],
 [1, 2, 0, 307, 5, 307],
 [1, 1, 4, 475],
 [1, 1, 4, 487],
 [1, 1, 4, 395],
 [1, 1, 0, 468],
 [1, 1, 0, 603],
 [1, 1, 4, 517],
 [1, 1, 0, 502],
 [1, 1, 4, 293],
 [1, 1, 0, 463],
 [1, 1, 0, 531],
 [1, 2, 0, 576, 1, 576],
 [1, 1, 4, 411],
 [1, 1, 0, 655],
 [1, 1, 0, 328],
 [1, 1, 4, 385],
 [1, 1, 0, 335],
 [1, 2, 0, 425, 2, 425],
 [1, 1, 0, 438],
 [1, 1, 4, 377],
 [1, 3, 0, 378, 1, 138, 2, 378],
 [1, 2, 0, 369, 3, 369],
 [1, 2, 0, 437, 1, 437],
 [1, 1, 4, 329],
 [1, 1, 0, 444],
 [1, 3, 0, 763, 1, 763, 6, 763],
 [1, 1, 0, 666],
 [1, 2, 0, 335, 2, 335],
 [1, 1, 0, 523],
 [1, 2, 0, 678, 6, 678],
 [1, 2, 0, 287, 5, 47],
 [1, 3, 0, 629, 1, 629, 6, 629],
 [1, 1, 4, 507],
 [1, 1, 4, 432],
 [1, 1, 0, 756],
 [1, 2, 0, 289, 5, 289],
 [1, 2, 0, 700, 6, 700],
 [1, 2, 0, 329, 3, 329],
 [1, 1, 4, 541]]

In [53]:
len(matched_data)

40

## Data preparation

In [57]:
with open(PREPARED_DATA_PATH + PREPARED_DAY.replace('json', 'fjs'), 'w') as f:
    f.write(str(jobs.shape[0]) + ' ' + str(ovens.shape[0]) + ' ' + '1''\n')

In [58]:
with open(PREPARED_DATA_PATH + PREPARED_DAY.replace('json', 'fjs'), 'a') as file:
    for data in matched_data:
        data = list(map(str, data))
        file.write(' '.join(data) + '\n') 

## Pipeline

In [24]:
def getdata(filename='/content/day-99.fjs'):
    f=open(filename,'r')
    line=f.readline()
    print(line)
    line_data=line.strip().split()

    numbers_float=list(map(float,line_data))

    n=int(numbers_float[0])
    m=int(numbers_float[1])
    #average_num_machine=numbers_float[2]
    #print(n)
    #print(m)
    operations_machines={}#the available machines for each operation of jobs
    operations_times={}#the processing time for each operation of jobs


    # jobs=[[]for i in range(n)]
    numonJobs=[]
    # print(jobs)
    for i in range(n):
        line=f.readline()
        line_data=line.split()
        numbers_float = list(map(int, line_data))
        operation_num=int(numbers_float[0])
        numonJobs.append(operation_num)
        # operations=[[] for j in range(operation_num)]
        jj=1
        j=0
        while jj<len(numbers_float):
            o_num=int(numbers_float[jj])
            job_op=[]
            job_machines=[]
            job_processingtime=[]
            for kk in range(0,o_num*2,2):
                # job_op.append(numbers_float[jj+kk+1])
                job_machines.append(numbers_float[jj+kk+1])
                job_processingtime.append(numbers_float[jj+kk+1+1])
            # operations[j]=job_op
            operations_machines[(i+1,j+1)]=job_machines
            for l in range(len(job_machines)):
                operations_times[(i + 1, j + 1,job_machines[l])] = job_processingtime[l]

            j+=1
            jj+=o_num*2+1
    f.close()  # close the file
        # jobs[i]=operations

    # print(operations_machines)
        # print(operations_times)
        # print(numonJobs)
    J=list(range(1,n+1)) #define the index of jobs
    M=list(range(1,m+1)) #define the index of machines
    OJ={}
    for j in range(n):
        OJ[(J[j])]=list(range(1,numonJobs[j]+1))
    print(OJ)
    #define large_M
    largeM=0
    for job in J:
        for op in OJ[(job)]:
            protimemax=0
            for l in operations_machines[(job,op)]:
                if protimemax<operations_times[(job,op,l)]:
                    protimemax=operations_times[(job,op,l)]
            largeM+=protimemax


    Data={
        'n':n,
        'm':m,
        'J':J,
        'M':M,
        'OJ':OJ,
        'operations_machines':operations_machines,
        'operations_times':operations_times,
        'largeM':largeM,}
    return Data

In [25]:
filename = PREPARED_DATA_PATH + 'day-99.fjs'
Data = getdata(filename)

40 7

{1: [1], 2: [1], 3: [1], 4: [1], 5: [1], 6: [1], 7: [1], 8: [1], 9: [1], 10: [1], 11: [1], 12: [1], 13: [1], 14: [1], 15: [1], 16: [1], 17: [1], 18: [1], 19: [1], 20: [1], 21: [1], 22: [1], 23: [1], 24: [1], 25: [1], 26: [1], 27: [1], 28: [1], 29: [1], 30: [1], 31: [1], 32: [1], 33: [1], 34: [1], 35: [1], 36: [1], 37: [1], 38: [1], 39: [1], 40: [1]}


In [26]:
Data['operations_machines']

{(1, 1): [4],
 (2, 1): [0, 5],
 (3, 1): [4],
 (4, 1): [4],
 (5, 1): [4],
 (6, 1): [0],
 (7, 1): [0],
 (8, 1): [4],
 (9, 1): [0],
 (10, 1): [4],
 (11, 1): [0],
 (12, 1): [0],
 (13, 1): [0, 1],
 (14, 1): [4],
 (15, 1): [0],
 (16, 1): [0],
 (17, 1): [4],
 (18, 1): [0],
 (19, 1): [0, 2],
 (20, 1): [0],
 (21, 1): [4],
 (22, 1): [0, 1, 2],
 (23, 1): [0, 3],
 (24, 1): [0, 1],
 (25, 1): [4],
 (26, 1): [0],
 (27, 1): [0, 1, 6],
 (28, 1): [0],
 (29, 1): [0, 2],
 (30, 1): [0],
 (31, 1): [0, 6],
 (32, 1): [0, 5],
 (33, 1): [0, 1, 6],
 (34, 1): [4],
 (35, 1): [4],
 (36, 1): [0],
 (37, 1): [0, 5],
 (38, 1): [0, 6],
 (39, 1): [0, 3],
 (40, 1): [4]}

In [27]:
def MIPModel(Data):
    from gurobipy import Model, GRB, quicksum
    import sys

    n = Data['n']
    m = Data['m']
    J = Data['J']
    OJ = Data['OJ']
    operations_machines = Data['operations_machines']
    operations_times = Data['operations_times']
    largeM = Data['largeM']

    # preprocessing
    # obtain the earliest starting time for each operation
    stimeOp = {}
    stimeOpMax = 0
    for i in J:
        for j in OJ[i]:
            if j == 1:
                stimeOp[i, j] = 0
            else:
                opt_time_min = sys.maxsize
                for k in operations_machines[i, j - 1]:
                    if operations_times[i, j - 1, k] < opt_time_min:
                        opt_time_min = operations_times[i, j - 1, k]
                stimeOp[i, j] = stimeOp[i, j - 1] + opt_time_min
            if stimeOpMax < stimeOp[i, j]:
                stimeOpMax = stimeOp[i, j]
    # print(stimeOp)
    # the latest completion time for each operation
    ltimeOp = {}
    for i in J:
        for jj in range(len(OJ[i]) - 1, -1, -1):
            j = OJ[i][jj]
            opt_time_min = sys.maxsize
            for k in operations_machines[i, j]:
                if operations_times[i, j, k] < opt_time_min:
                    opt_time_min = operations_times[i, j, k]

            if jj == len(OJ[i]) - 1:
                ltimeOp[i, j] = largeM - opt_time_min
            else:
                ltimeOp[i, j] = ltimeOp[i, j + 1] - opt_time_min
    print(ltimeOp)

    model = Model("FJSP_PPF")

    x, y, s = {}, {}, {}
    cmax = model.addVar(lb=stimeOpMax, ub=largeM, vtype="I", name="cmax")
    # define x
    for j in J:  # job
        for i in OJ[j]:  # operation
            # s[j,i]=model.addVar(lb=0,ub=largeM,vtype="I",name="s(%s,%s)"%(j,i))
            s[j, i] = model.addVar(lb=stimeOp[j, i], ub=largeM, vtype="I", name="s(%s,%s)" % (j, i))  # ltimeOp[j,i]
            for k in operations_machines[j, i]:
                x[j, i, k] = model.addVar(lb=0, ub=1, vtype="I", name="x(%s,%s,%s)" % (j, i, k))
    # define y
    for i in J:
        for ip in J:
            for j in OJ[i]:
                for jp in OJ[ip]:
                    y[i, j, ip, jp] = model.addVar(lb=0, ub=1, vtype="I", name="y(%s,%s,%s,%s)" % (i, j, ip, jp))

    # define objective function
    model.setObjective(cmax, GRB.MINIMIZE)
    # constraint(3)
    for i in J:
        for j in OJ[i]:
            model.addConstr(quicksum(x[i, j, k] for k in operations_machines[i, j]) == 1, "assignment(%s,%s)" % (i, j))
    # constraint(4)
    for i in J:
        for j in OJ[i]:
            if j != OJ[i][0]:
                model.addConstr(s[i, j] >= s[i, j - 1] + quicksum(
                    operations_times[i, j - 1, k] * x[i, j - 1, k] for k in operations_machines[i, j - 1]),
                                "stime(%s,%s)" % (i, j))
    # constraint(5)
    for i in J:
        for ip in J:
            if i < ip:
                for j in OJ[i]:
                    for jp in OJ[ip]:
                        kkp = [k for k in operations_machines[i, j] if k in operations_machines[ip, jp]]
                        if len(kkp):
                            for iipk in kkp:
                                model.addConstr(s[ip, jp] >= s[i, j] + operations_times[i, j, iipk] - largeM * (
                                            3 - y[i, j, ip, jp] - x[i, j, iipk] - x[ip, jp, iipk]),
                                                "cons_6_(%s,%s,%s,%s)" % (i, j, ip, jp))
                                model.addConstr(
                                    s[i, j] >= s[ip, jp] +
                                    operations_times[ip, jp, iipk] - largeM *
                                    (2 + y[i, j, ip, jp] - x[i, j, iipk] - x[ip, jp, iipk]),
                                    "cons_5_(%s,%s,%s,%s)" % (i, j, ip, jp))
    # cmax constraint
    for i in J:
        model.addConstr(cmax >= s[i, OJ[i][len(OJ[i]) - 1]] + quicksum(
            operations_times[i, OJ[i][len(OJ[i]) - 1], k] * x[i, OJ[i][len(OJ[i]) - 1], k] for k in
            operations_machines[i, OJ[i][len(OJ[i]) - 1]]), "cmax_cons_(%s)" % (i))



    model.params.TimeLimit=1200
    model.update()
    return model

In [28]:
import numpy as np
import time
import os

filename=PREPARED_DATA_PATH + 'day-99.fjs'
Data=getdata(filename)
print('data_j',Data['J'],Data['OJ'])
print('DATA_operations_machines',Data['operations_machines'])
print('operations_times',Data['operations_times'])

num_operation = []
for i in Data['J']:
    num_operation.append(Data['OJ'][i][-1])
print(num_operation)
num_operation_max = np.array(num_operation).max()

time_window = np.zeros(shape=(Data['n'],num_operation_max,Data['m']))

for i in range(Data['n']):
    for j in range(Data['m']):
        try:
            mchForJob = Data['operations_machines'][(i,j)]
            for k in mchForJob:
                time_window[i][j][k-1] = Data['operations_times'][(i+1,j+1,k)]
        except:
            pass
print(time_window)

n=Data['n']
m=Data['m']
J=Data['J']
OJ=Data['OJ']
operations_machines=Data['operations_machines']
operations_times=Data['operations_times']
largeM=Data['largeM']

mipmodel=MIPModel(Data)
mipmodel.optimize()

40 7

{1: [1], 2: [1], 3: [1], 4: [1], 5: [1], 6: [1], 7: [1], 8: [1], 9: [1], 10: [1], 11: [1], 12: [1], 13: [1], 14: [1], 15: [1], 16: [1], 17: [1], 18: [1], 19: [1], 20: [1], 21: [1], 22: [1], 23: [1], 24: [1], 25: [1], 26: [1], 27: [1], 28: [1], 29: [1], 30: [1], 31: [1], 32: [1], 33: [1], 34: [1], 35: [1], 36: [1], 37: [1], 38: [1], 39: [1], 40: [1]}
data_j [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] {1: [1], 2: [1], 3: [1], 4: [1], 5: [1], 6: [1], 7: [1], 8: [1], 9: [1], 10: [1], 11: [1], 12: [1], 13: [1], 14: [1], 15: [1], 16: [1], 17: [1], 18: [1], 19: [1], 20: [1], 21: [1], 22: [1], 23: [1], 24: [1], 25: [1], 26: [1], 27: [1], 28: [1], 29: [1], 30: [1], 31: [1], 32: [1], 33: [1], 34: [1], 35: [1], 36: [1], 37: [1], 38: [1], 39: [1], 40: [1]}
DATA_operations_machines {(1, 1): [4], (2, 1): [0, 5], (3, 1): [4], (4, 1): [4], (5, 1): [4], (6, 1): [0], (7, 1): [0], (8, 1): [4]

  4144  1587 infeasible   39      6712.00000 3151.60388  53.0%  37.5    5s
 10098  4111 infeasible   39      6712.00000 3254.16378  51.5%  36.5   10s
 13050  5196 3508.00733   19  131 6712.00000 3291.69309  51.0%  35.2   15s
 16393  6638 4230.00000   26   49 6712.00000 3330.17553  50.4%  33.5   20s
 21559  8468 4367.10539   23   68 6712.00000 3372.61160  49.8%  31.6   31s
 25319 10248 3592.00000   27   78 6712.00000 3408.00000  49.2%  30.7   35s
 29772 11374 4641.00000   27  124 6712.00000 3440.00000  48.7%  30.0   43s
 29781 11380 4642.00000   33  143 6712.00000 3440.00000  48.7%  30.0   45s
 29806 11397 4120.00000   24  147 6712.00000 3440.00000  48.7%  29.9   50s
 29836 11417 5751.00000   37  133 6712.00000 3440.00000  48.7%  29.9   55s
 29870 11439 4094.53418   26  133 6712.00000 3440.00000  48.7%  29.9   60s
 29900 11459 5875.00000   36  126 6712.00000 3440.00000  48.7%  29.8   65s
 29930 11479 3653.00000   24  126 6712.00000 3440.00000  48.7%  29.8   70s
 29958 11498 5861.00000  

 285405 84944 5025.00000   46   48 6712.00000 3588.29984  46.5%  17.8  556s
 287285 85498 4730.00000   47   37 6712.00000 3589.00000  46.5%  17.7  562s
 289041 86042 4214.00000   49   59 6712.00000 3590.00000  46.5%  17.7  566s
 290910 86568 4040.00000   40   56 6712.00000 3591.00000  46.5%  17.7  572s
 292621 87076 5921.00000   51   42 6712.00000 3591.72328  46.5%  17.7  576s
 294365 87633 infeasible   56      6712.00000 3592.97416  46.5%  17.6  581s
 296073 88175 infeasible   50      6712.00000 3593.56781  46.5%  17.6  586s
 298897 88884 4309.00000   38   52 6712.00000 3594.95060  46.4%  17.5  590s
 301612 89609 5502.00000   49   51 6712.00000 3596.00000  46.4%  17.5  596s
 304091 90278 3921.00000   42   60 6712.00000 3597.00000  46.4%  17.4  601s
 305647 90764 4518.00000   50   30 6712.00000 3598.00000  46.4%  17.4  605s
 308029 91489 infeasible   43      6712.00000 3599.21429  46.4%  17.4  610s
 310484 92261 3672.00000   53   65 6712.00000 3601.00000  46.3%  17.4  616s
 312097 9278

 572849 168433 3914.00000   47   64 6712.00000 3702.00000  44.8%  14.8 1090s
 579141 170099 5874.00000   51   26 6712.00000 3704.00000  44.8%  14.8 1095s
 587087 172332 3870.00000   43   49 6712.00000 3706.00000  44.8%  14.7 1100s
 590755 173272 5055.00000   45   43 6712.00000 3707.00000  44.8%  14.7 1115s
 595252 174823 4237.00000   44   47 6712.00000 3708.76891  44.7%  14.7 1120s
 601129 176431 5280.00000   50   41 6712.00000 3710.46883  44.7%  14.6 1125s
 606511 177897 4233.00000   43   64 6712.00000 3712.00000  44.7%  14.6 1130s
 611973 179518 4976.00000   43   43 6712.00000 3714.00000  44.7%  14.6 1135s
 617745 180836 4365.00000   44   53 6712.00000 3715.00000  44.7%  14.5 1145s
 620983 182089 infeasible   56      6712.00000 3716.00000  44.6%  14.5 1150s
 627148 183906 infeasible   54      6712.00000 3718.04101  44.6%  14.5 1155s
 632066 185208 5383.00000   48   34 6712.00000 3720.00000  44.6%  14.5 1160s
 637502 186802 4762.00000   43   45 6712.00000 3721.07143  44.6%  14.4 1165s

In [29]:
mipmodel.X

[6712.0,
 2515.0,
 1.0,
 336.0,
 0.0,
 1.0,
 3279.0,
 1.0,
 678.0,
 1.0,
 6317.0,
 1.0,
 1279.0,
 1.0,
 3131.0,
 1.0,
 4083.0,
 1.0,
 1747.0,
 1.0,
 0.0,
 1.0,
 4593.0,
 1.0,
 4062.0,
 1.0,
 1200.0,
 0.0,
 1.0,
 2104.0,
 1.0,
 5391.0,
 1.0,
 3734.0,
 1.0,
 293.0,
 1.0,
 5056.0,
 1.0,
 713.0,
 0.0,
 1.0,
 2249.0,
 1.0,
 2902.0,
 1.0,
 335.0,
 0.0,
 0.0,
 1.0,
 329.0,
 0.0,
 1.0,
 763.0,
 0.0,
 1.0,
 3754.0,
 1.0,
 2687.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 6046.0,
 1.0,
 0.0,
 0.0,
 1.0,
 756.0,
 1.0,
 629.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1597.0,
 1.0,
 1165.0,
 1.0,
 0.0,
 1.0,
 47.0,
 0.0,
 1.0,
 1307.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 4600.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.